<a href="https://colab.research.google.com/github/bootneck2000/Yolo_FCNN_dev_pengos/blob/main/FasterRCNN_Roldan_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

# Example: If you have 3 classes (excluding background), use 4
num_classes = 2  # <-- CHANGE this to match your training


import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# (Re)define your get_fasterrcnn_model function if not already defined:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_fasterrcnn_model(num_classes):
    model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

model_path = "/content/drive/My Drive/fasterrcnn_roboflow_yolov8.pth"
model = get_fasterrcnn_model(num_classes)
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()

from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_fasterrcnn_model(num_classes):
    model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

model_path = '/content/drive/MyDrive/fasterrcnn_roboflow_yolov8.pth'

# Recreate your model architecture
model = get_fasterrcnn_model(num_classes)  # Make sure num_classes matches training
model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()
model.to(device)  # Use your device (cuda/cpu)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [19]:
def download_and_extract_roboflow_zip(roboflow_zip_url, extract_to="roboflow_dataset"):
    if os.path.exists(extract_to):
        print(f"'{extract_to}' already exists, skipping download.")
        return
    print("Downloading dataset from Roboflow...")
    r = requests.get(roboflow_zip_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(extract_to)
    print(f"Dataset extracted to '{extract_to}'.")

# Cell 4: YOLOv8 to COCO box conversion (helper)
def yolo_to_coco(bbox, img_w, img_h):
    # bbox: [x_center, y_center, width, height] normalized
    x_c, y_c, w, h = bbox
    x_c *= img_w
    y_c *= img_h
    w *= img_w
    h *= img_h
    # Convert to [xmin, ymin, xmax, ymax]
    xmin = x_c - w / 2
    ymin = y_c - h / 2
    xmax = x_c + w / 2
    ymax = y_c + h / 2
    return [xmin, ymin, xmax, ymax]

# Cell 5: Roboflow YOLO8 Dataset
import os
from torch.utils.data import Dataset
from PIL import Image
import torch
from torchvision.transforms import functional as F

class RoboflowYolo8Dataset(Dataset):
    def __init__(self, data_dir, split="train", transforms=None):
        # ... (directory finding code as before) ...
        self.images_dir = os.path.join(data_dir, split, "images")
        self.labels_dir = os.path.join(data_dir, split, "labels")
        self.img_filenames = [f for f in os.listdir(self.images_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        self.transforms = transforms

        # Build class_dict from all label files
        self.class_dict = self._find_classes()

    def _find_classes(self):
        classes = set()
        for label_file in os.listdir(self.labels_dir):
            with open(os.path.join(self.labels_dir, label_file)) as f:
                for line in f:
                    if line.strip():
                        parts = line.strip().split()
                        classes.add(int(parts[0]))
        sorted_classes = sorted(list(classes))
        return {idx: idx for idx in sorted_classes}  # assumes contiguous class ids

    def __getitem__(self, idx):
        # ... (rest of your code, as in previous examples) ...
        img_name = self.img_filenames[idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_name = os.path.splitext(img_name)[0] + ".txt"
        label_path = os.path.join(self.labels_dir, label_name)

        image = Image.open(img_path).convert("RGB")
        w, h = image.size

        boxes = []
        labels = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    if line.strip():
                        parts = line.strip().split()
                        cls, xc, yc, bw, bh = map(float, parts[:5])
                        xc, yc, bw, bh = xc * w, yc * h, bw * w, bh * h
                        x1 = xc - bw/2
                        y1 = yc - bh/2
                        x2 = xc + bw/2
                        y2 = yc + bh/2
                        boxes.append([x1, y1, x2, y2])
                        labels.append(int(cls))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        if boxes.numel() == 0:
            return None

        target = {"boxes": boxes, "labels": labels, "image_id": torch.tensor([idx])}
        if self.transforms:
            image = self.transforms(image)
        else:
            image = F.to_tensor(image)
        return image, target

    def __len__(self):
        return len(self.img_filenames)

def __getitem__(self, idx):
    img_name = self.img_filenames[idx]
    img_path = os.path.join(self.images_dir, img_name)
    label_name = os.path.splitext(img_name)[0] + ".txt"
    label_path = os.path.join(self.labels_dir, label_name)

    image = Image.open(img_path).convert("RGB")
    w, h = image.size

    boxes = []
    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                if line.strip():
                    parts = line.strip().split()
                    cls, xc, yc, bw, bh = map(float, parts)
                    xc, yc, bw, bh = xc * w, yc * h, bw * w, bh * h
                    x1 = xc - bw / 2
                    y1 = yc - bh / 2
                    x2 = xc + bw / 2
                    y2 = yc + bh / 2
                    boxes.append([x1, y1, x2, y2])
                    labels.append(int(cls))

    boxes = torch.tensor(boxes, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.int64)

    if boxes.numel() == 0:   # <-- FIX: Check boxes before using target!
        return None

    target = {"boxes": boxes, "labels": labels, "image_id": torch.tensor([idx])}

    if self.transforms:
        image = self.transforms(image)
    else:
        image = F.to_tensor(image)
    return image, target

# Cell 6: Helper for collate_fn (required by DataLoader)
def collate_fn(batch):
    return tuple(zip(*batch))

In [32]:
import requests
import zipfile
import os

# Path where your Roboflow test images are stored
ROBOFLOW_TEST_IMAGES = '/content/roboflow_dataset/test/images'
# Path to your trained model
MODEL_PATH = '/content/drive/My Drive/fasterrcnn_roboflow_yolov8.pth'
# Folder to save results
RESULTS_DIR = '/content/fasterrcnn_test_results2'

os.makedirs(RESULTS_DIR, exist_ok=True)

# Download and extract Roboflow ZIP
#ROBOFLOW_ZIP_URL = "https://app.roboflow.com/ds/VquXPr9PVz?key=QyrlUzVsjR"
ROBOFLOW_ZIP_URL = "https://app.roboflow.com/ds/sU8uW5aryJ?key=zytgPCj2cT"
DATASET_DIR = "roboflow_dataset"
ZIP_PATH = "roboflow.zip"

def download_and_extract_roboflow_zip(url, extract_to):
    # Download the dataset
    print("Downloading dataset...")
    r = requests.get(url)
    with open(ZIP_PATH, "wb") as f:
        f.write(r.content)
    # Extract the zip file
    print("Extracting dataset...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Done.")

download_and_extract_roboflow_zip(ROBOFLOW_ZIP_URL, DATASET_DIR)

Extracting dataset...
Done.


In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import torchvision.transforms as T
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# Adjust this according to the number of classes you trained with (+1 for background)
NUM_CLASSES = 2  # Change to match your model (e.g. 2 for 1 class + background)

def get_fasterrcnn_model(num_classes):
    model = fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model
model = get_fasterrcnn_model(NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval().to(device)

In [37]:
# Prepare data
image_files = [f for f in os.listdir(ROBOFLOW_TEST_IMAGES) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
transform = T.ToTensor()

# Prepare CSV for results
results_list = []

for image_name in tqdm(image_files):
    img_path = os.path.join(ROBOFLOW_TEST_IMAGES, image_name)
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)[0]

    boxes = outputs['boxes'].cpu().numpy()
    labels = outputs['labels'].cpu().numpy()
    scores = outputs['scores'].cpu().numpy()

    # Save annotated image
    np_img = np.array(img)
    for box, label, score in zip(boxes, labels, scores):
        if score > 0.65:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(np_img, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(np_img, f'{label}:{score:.2f}', (x1, max(0, y1-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,255,0), 1)
            # Save to CSV
            results_list.append([image_name, x1, y1, x2, y2, int(label), float(score)])
    save_path = os.path.join(RESULTS_DIR, f'pred_{image_name}')
    cv2.imwrite(save_path, cv2.cvtColor(np_img, cv2.COLOR_RGB2BGR))

# Save CSV
results_df = pd.DataFrame(results_list, columns=["image", "x1", "y1", "x2", "y2", "label", "score"])
results_df.to_csv(os.path.join(RESULTS_DIR, 'inference_results.csv'), index=False)

print(f"Results saved in {RESULTS_DIR}")

  2%|▏         | 8/459 [01:40<1:34:38, 12.59s/it]


KeyboardInterrupt: 


KeyboardInterrupt



In [ ]:
!zip -r /content/fasterrcnn_test_results.zip /content/fasterrcnn_test_results
from google.colab import files
files.download('/content/fasterrcnn_test_results.zip')